In [29]:
import os
import re
import shutil
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image, ImageOps

In [30]:
# 추론한 이미지들을 담고 있는 디렉토리 경로
processed_dataset_dir = './processed_dataset'

# 학습할 마스크 이미지 경로
mask_dataset_dir = './mask_dataset'

# 다시 박스 쳐서 추론할 이미지 경로
recheck_image_dir = './recheck_image'

# 학습할 마스크 이미지 리스트
mask_list = []

# 다시 박스 쳐서 추론할 이미지 리스트
recheck_image_list = []

In [31]:
# 이미지를 담고 있는 디렉토리 클래스 구조
class image_dir:
    def __init__(self, image_dir_path, original, mask1_origin, mask1, mask2_origin, mask2, mask3_origin, mask3):

        # 이미지들을 담고 있는 디렉토리 경로
        self.image_dir_path = image_dir_path

        # 원본 이미지 이름
        self.original = original

        # 1번 마스크
        self.mask1_origin = mask1_origin
        self.mask1 = mask1

        # 2번 마스크
        self.mask2_origin = mask2_origin
        self.mask2 = mask2

        # 3번 마스크
        self.mask3_origin = mask3_origin
        self.mask3 = mask3

        # 이터레이터를 위한 리스트 초기화
        self.list = [
            self.original,
            self.mask1_origin, self.mask1,
            self.mask2_origin, self.mask2,
            self.mask3_origin, self.mask3
        ]
        
    def __iter__(self):
        # 이터레이터 초기화
        self._iter_index = 0
        return self

    def __next__(self):
        if self._iter_index < len(self.list):
            result = self.list[self._iter_index]
            self._iter_index += 1
            return result
        else:
            raise StopIteration
    
     # 이미지 불러오는 함수
    def open_image(self, image_name):
        if self.image_dir_path is None or image_name is None:
            raise ValueError("image_dir_path 또는 image_name이 None입니다.")
    
        image_path = os.path.join(self.image_dir_path, image_name)
        img = Image.open(image_path)
        img = ImageOps.exif_transpose(img)
        return img

In [32]:
# 이미지 담고 있는 디렉토리 리스트 초기화
image_dir_list = []

# 이미지 담고 있는 디렉토리들을 정리
for image_dir_name in os.listdir(processed_dataset_dir):
    image_dir_path = os.path.join(processed_dataset_dir, image_dir_name)

    original = None
    original_path = None
    
    # 이미지 담고 있는 디렉토리 열기
    for file in os.listdir(image_dir_path):

        if file.lower().endswith('.jpg'):
            file_path = os.path.join(image_dir_path, file)
            # 원본 이미지
            if original is None or os.path.getctime(file_path) > os.path.getctime(original_path):
                original = file
                original_path = file_path

        # 마스크 처리
        elif file.lower().endswith('.png'):
            parts = re.split(r'[_\.]', file)
            if parts[1] == '1':
                if parts[2] == 'origin':
                    mask1_origin = file
                else:
                    mask1 = file
            elif parts[1] == '2':
                if parts[2] == 'origin':
                    mask2_origin = file
                else:
                    mask2 = file
            elif parts[1] == '3':
                if parts[2] == 'origin':
                    mask3_origin = file
                else:
                    mask3 = file

    # 이미지 담고 있는 디렉토리 리스트에 추가
    image_dir_list.append(image_dir(
        image_dir_path=image_dir_path, 
        original=original, 
        mask1_origin=mask1_origin, 
        mask1=mask1, 
        mask2_origin=mask2_origin, 
        mask2=mask2, 
        mask3_origin=mask3_origin, 
        mask3=mask3
        ))

In [33]:
# 이미지 디스플레이 함수
def show_image(index):
    clear_output(wait=True)
    
    if index+1 <= len(image_dir_list):
        img_dir = image_dir_list[index]
        
        # 현재 인덱스 표시
        print(index)

        # 디버깅 출력
        print(f"Image Directory Path: {img_dir.image_dir_path}")
        
        # figure 사이즈 설정
        plt.figure(figsize=(10, 10))

        # 이미지 담고 있는 디렉토리 열어서 이미지 불러오기
        for i, img_name in enumerate(img_dir):
            img = img_dir.open_image(img_name)
            
            # 원본 이미지 따로 표시
            if i == 0:
                print(f"Image Name: {i}: {img_name}")
                plt.subplot(4, 2, i+1)
                plt.imshow(img)
                plt.axis('off')
                # plt.title(img)
            else:
                print(f"Image Name: {i}: {img_name}")
                plt.subplot(4, 2, i+2)
                plt.imshow(img)
                plt.axis('off')
                # plt.title(img)
                
        plt.show()

    else:
        print("모든 이미지를 이동했습니다.")

    display(mask1_button, mask2_button, mask3_button, recheck_image_button, prev_button, next_button, remove_mask_button, remove_image_button)

In [34]:
# 이전 이미지 디렉토리로 이동하는 함수
def prev_image(b):
    global index
    # index = (index - 1) % len(image_dir_list)
    index = index - 1
    show_image(index)

# 다음 이미지 디렉토리로 이동하는 함수
def next_image(b):
    global index
    # index = (index + 1) % len(image_dir_list)
    index = index + 1
    show_image(index)

In [35]:
# 1번 마스크 복사하는 함수
def copy_mask1_next(b):
    global index
    if image_dir_list:
        img_dir = image_dir_list[index]

        # 1번 마스크 복사
        source_path = os.path.join(img_dir.image_dir_path, img_dir.mask1)
        target_path = os.path.join(mask_dataset_dir, img_dir.mask1)
        shutil.copy(source_path, target_path)

        # 옮긴 마스크 경로 리스트에 등록
        mask_list.append(target_path)

        # index = (index + 1) % len(image_dir_list)
        index = index + 1
        show_image(index)

# 2번 마스크 복사하는 함수
def copy_mask2_next(b):
    global index
    if image_dir_list:
        img_dir = image_dir_list[index]

        # 2번 마스크 복사
        source_path = os.path.join(img_dir.image_dir_path, img_dir.mask2)
        target_path = os.path.join(mask_dataset_dir, img_dir.mask2)
        shutil.copy(source_path, target_path)

        # 옮긴 마스크 경로 리스트에 등록
        mask_list.append(target_path)

        # index = (index + 1) % len(image_dir_list)
        index = index + 1
        show_image(index)

# 3번 마스크 복사하는 함수
def copy_mask3_next(b):
    global index
    if image_dir_list:
        img_dir = image_dir_list[index]

        # 3번 마스크 복사
        source_path = os.path.join(img_dir.image_dir_path, img_dir.mask3)
        target_path = os.path.join(mask_dataset_dir, img_dir.mask3)
        shutil.copy(source_path, target_path)

        # 옮긴 마스크 경로 리스트에 등록
        mask_list.append(target_path)

        # index = (index + 1) % len(image_dir_list)
        index = index + 1
        show_image(index)

# 옮긴 마스크 삭제하는 함수
def remove_mask_previous(b):
    global index
    if mask_list:
        target_path = mask_list.pop()
        os.remove(target_path)
        
        # index = (index - 1) % len(image_dir_list)
        index = index - 1
        show_image(index)

In [36]:
# 다시 박스 쳐서 추론할 이미지 복사하는 함수
def copy_recheck_image_next(b):
    global index
    if image_dir_list:
        img_dir = image_dir_list[index]

        # recheck 이미지 복사
        source_path = os.path.join(img_dir.image_dir_path, img_dir.original)
        target_path = os.path.join(recheck_image_dir, img_dir.original)
        shutil.copy(source_path, target_path)

        # recheck 이미지 경로 리스트에 등록
        recheck_image_list.append(target_path)

        # index = (index + 1) % len(image_dir_list)
        index = index + 1
        show_image(index)
    
# recheck 이미지 삭제하는 함수
def remove_recheck_image_previous(b):
    global index
    if recheck_image_list:
        target_path = recheck_image_list.pop()
        os.remove(target_path)

        # index = (index - 1) % len(image_dir_list)
        index = index - 1
        show_image(index)

In [38]:
# 버튼 생성
prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")
mask1_button = widgets.Button(description="Mask1")
mask2_button = widgets.Button(description="Mask2")
mask3_button = widgets.Button(description="Mask3")
recheck_image_button = widgets.Button(description="Recheck Image")
remove_mask_button = widgets.Button(description="Remove Mask")
remove_image_button = widgets.Button(description="Remove Image")

# 버튼 클릭 이벤트에 함수 연결
prev_button.on_click(prev_image)
next_button.on_click(next_image)
mask1_button.on_click(copy_mask1_next)
mask2_button.on_click(copy_mask2_next)
mask3_button.on_click(copy_mask3_next)
recheck_image_button.on_click(copy_recheck_image_next)
remove_mask_button.on_click(remove_mask_previous)
remove_image_button.on_click(remove_recheck_image_previous)


# 인덱스 설정
index = 0

show_image(index)

모든 이미지를 이동했습니다.


Button(description='Mask1', style=ButtonStyle())

Button(description='Mask2', style=ButtonStyle())

Button(description='Mask3', style=ButtonStyle())

Button(description='Recheck Image', style=ButtonStyle())

Button(description='Previous', style=ButtonStyle())

Button(description='Next', style=ButtonStyle())

Button(description='Remove Mask', style=ButtonStyle())

Button(description='Remove Image', style=ButtonStyle())